## 머신러닝 실습

### 결정트리 분석

#### 개념
- if/else 기반으로 규칙을 찾아 학습, 트리 구조의 분류규칙을 만드는 것
  
##### 정보이득지수
- 엔트로피 개념을 기반으로 함. 엔트로피는 데이터 집합의 혼잡도를 뜻함.
  - 데이터 집합에 다른 데이터가 섞여서 균일도가 떨어지면 혼잡도가 높아진다고 함
  - 균일도가 높아지면 혼잡도가 낮아진다고 함
- 정보이득지수는 혼잡도가 줄어들어 얻게 되는 이득을 뜻함. **1-엔트로피로** 계산

##### 지니계수
- 데이터의 순도를 뜻함. 
  - 지니계수가 높을 수록 순도가 낮은 데이터 집함.
  - 지니게수가 0이면 완전 순수한 데이터 집합

In [ ]:
## 필요라이브러리 사용등록
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo


#### 데이터 수집

##### UC 어바인 ML연구소 스마트폰 센서데이터
- 웹사이트에서 다운로드
  - https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones
- 30명의 연구 자원 봉사자가 허리에 갤럭시 S2를 착용하고, 6가지 움직임을 수행, 수집한 XYZ 3축에 대한 데이터
- 움직임 - WALKING WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING

In [ ]:
# 피처이름
dfFeatureName = pd.read_csv('./data/features.txt', sep='\s+',header = None, names=['index','feature_name'],engine='python')

In [ ]:
dfFeatureName.tail()

In [ ]:
# index제거,
featureNames = dfFeatureName.iloc[:,1].values.tolist()

In [ ]:
## X_train, X_test, y_train, y_test 데이터로드
X_train = pd.read_csv('./data/train/X_train.txt',engine = 'python',sep='\s+',header = None)

In [ ]:
X_test = pd.read_csv('./data/test/X_test.txt',engine = 'python',sep='\s+',header = None)

In [ ]:
y_train = pd.read_csv('./data/train/y_train.txt',engine = 'python',sep='\s+',header = None)

In [ ]:
y_test = pd.read_csv('./data/test/y_test.txt',engine = 'python',sep='\s+',header = None)

In [ ]:
X_train.columns = featureNames

In [ ]:
X_train.tail()

In [ ]:
X_test.columns = featureNames

In [ ]:
X_test.tail()

In [ ]:
y_train.columns=['action']

In [ ]:
y_train.tail()

In [ ]:
y_test.columns = ['action']

In [ ]:
y_test.tail()

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
## Action의 개수
y_train.value_counts()

In [ ]:
labelName = pd.read_csv('./data/activity_labels.txt',engine = 'python', sep = '\s+',header = None).iloc[:,1].tolist()

##### 분석모델 구축

In [ ]:
## 결정트리 라이브러리 사용등록
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# 모델생성
model = DecisionTreeClassifier(random_state=40)

In [ ]:
# 모델훈련
model.fit(X_train, y_train)

In [ ]:
## 예측
y_predict = model.predict(X_test)

##### 분석지표 확인

In [ ]:
## 분석지표용 라이브러리 사용등록
from sklearn.metrics import accuracy_score

In [ ]:
# 정확도 판단
accuracy_score(y_test, y_predict)

In [ ]:
model.get_params()

##### 정확도를 높이기

In [ ]:
## 추가 라이브러리 사용등록
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {'max_depth': [6,8,10,12,16,20,24]}
gridCV = GridSearchCV(model,param_grid = params, scoring='accuracy', cv=5,return_train_score=True)

In [ ]:
gridCV.fit(X_train,y_train)

In [ ]:
dfCVResult = pd.DataFrame(gridCV.cv_results_)

In [ ]:
dfCVResult[['param_max_depth','mean_test_score','mean_train_score']]

In [ ]:
## 하이퍼 파라미터를 변경
params = {'max_depth': [8,10,16], 'min_samples_split':[2,20,24]}

gridCV = GridSearchCV(model,param_grid = params, scoring='accuracy', cv=5,return_train_score=True)

In [ ]:
gridCV.fit(X_train,y_train)

In [ ]:
dfCVResult =pd.DataFrame(gridCV.cv_results_)
dfCVResult[['param_max_depth','param_min_samples_split','mean_test_score','mean_train_score']]

In [ ]:
gridCV.best_score_

In [ ]:
gridCV.best_params_

In [ ]:
gridCV.best_estimator_

In [ ]:
## 최고 예측치로 다시 예측
bestModel = gridCV.best_estimator_

In [ ]:
best_y_predict = bestModel.predict(X_test)

In [ ]:
## 정확도
accuracy_score(y_test,best_y_predict)

##### 시각화

In [ ]:
# 독립변수 561개 중 중요도 높은 변수를 찾는 것
len(bestModel.feature_importances_)

In [ ]:
featureImportances = bestModel.feature_importances_

In [ ]:
featureTop10 = pd.Series(featureImportances, index=X_train.columns).sort_values(ascending=False)[:10]

In [ ]:
featureTop10

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 바 차트
plt.figure(figsize= (15,5))
plt.title('Smarthone sensor feat TOP10')
sns.barplot(x=featureTop10, y = featureTop10.index, hue=featureTop10)
plt.show()